# Collaborative Filtering Recommendation System

## Import Modules

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics.pairwise import cosine_similarity


# Import Dataset

In [54]:
df1 = pd.read_csv(filepath_or_buffer="./Movie_data.csv", names=["user_id", "username", "item_id", "rating", "timestamp"])
df1["timestamp"] = pd.to_datetime(df1["timestamp"], unit="s")

df2 = pd.read_csv(filepath_or_buffer="./Movie_Id_Titles.csv")

join_df = df1.join(df2.set_index("item_id"), on="item_id")
join_df = join_df.drop(columns="item_id")

join_df.head()

,user_id,username,rating,timestamp,title
0,0,Shawn Wilson,5,1997-12-04 15:55:49,Star Wars (1977)
1,0,Shawn Wilson,5,1997-12-04 15:55:49,"Empire Strikes Back, The (1980)"
2,0,Shawn Wilson,1,1997-12-04 15:55:49,Gone with the Wind (1939)
3,196,Bessie White,3,1997-12-04 15:55:49,Kolya (1996)
4,196,Bessie White,4,1997-12-04 16:11:03,Mrs. Doubtfire (1993)
